In [2]:
import os
import gc
gc.collect()

0

In [ ]:
import pickle
with open('no_img_fake', 'rb') as file:
    no_img_fake = pickle.load(file)
    
with open('no_img_real', 'rb') as file:
    no_img_real = pickle.load(file)
    
with open('no_top_img_fake', 'rbzaidai') as file:
    no_top_img_fake = pickle.load(file)
    
with open('no_top_img_real', 'rb') as file:
    no_top_img_real = pickle.load(file)

In [ ]:
import pandas as pd
df_fake = pd.read_csv('politifact_fake.csv')
df_real = pd.read_csv('politifact_real.csv')

In [1]:

import numpy as np
import torch
import pandas as pd
from keras.layers import Dense
from keras.utils import to_categorical
from keras.models import Sequential
from nltk.tokenize import sent_tokenize
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import pickle as pickle

Using TensorFlow backend.


In [ ]:
# 从给定的虚假信息数据帧中筛选出满足特定条件的数据，然后构建一个新的数据帧
def make_dataframe(df_fake,no_img_fake,no_top_img_fake,labels):
    name=[]
    title=[]
    text=[]
    label=[]
    for i, r in df_fake.iterrows():
    #     print(type(r['name']))
        if(str(r['name']) in no_img_fake and str(r['name']) in no_top_img_fake):
            name.append(r['name'])
            title.append(r['title'])
            text.append(r['text'])
            label.append(labels)
    fake = pd.DataFrame()
    fake['name'] = name
    fake['title'] = title
    fake['text'] = text
    fake['label'] = label
    return fake

In [ ]:
fake = make_dataframe(df_fake,no_img_fake,no_top_img_fake,0)
real = make_dataframe(df_real,no_img_real,no_top_img_real,1)

In [ ]:
fake =fake.dropna()
real = real.dropna()

In [ ]:
# Fake Images 
# 创建一个字典 d，该字典以虚假信息的名称作为键，将与每个虚假信息相关的图像文件名存储为值。
import os
directory = 'politifact_fake_top_img'
file_names_top = os.listdir(directory)
file_names = os.listdir('fake_img')
d ={}
for i, r in fake.iterrows():
    name = r['name']
    d[name] = []
    for n in file_names_top:
        if name in n:
            d[name].append(n)
    for n in file_names:
        if name in n:
            d[name].append(n)
            
    

In [ ]:
# max_val 存储的是虚假信息的图像文件名列表长度的众数
import matplotlib.pyplot as plt
from scipy.stats import mode
histogram =[]
for key, value in d.items():
    l = len(value)
    histogram.append(l)
    
h= plt.hist(histogram, bins='auto')
plt.show()

max_val = mode(histogram)

In [3]:
# 将输入的图像文件处理成适合深度学习模型的格式，包括调整形状、转换为数组，并进行必要的预处理
def proc(img):
    
    from keras.preprocessing.image import load_img
    # load an image from file
    image = load_img(img, target_size=(224, 224))


    from keras.preprocessing.image import img_to_array
    # convert the image pixels to a numpy array
    image = img_to_array(image)


    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

    from keras.applications.vgg16 import preprocess_input
    # prepare the image for the VGG model
    image = preprocess_input(image)
    
    return image

In [ ]:
# ar 列表包含经过处理的图像文件名列表，其中长度要么是众数值，要么是整个原始列表
ar=[]
for key, value in d.items():
    if(len(value) > max_val):
        ar.append(value[:max_val])
    else:
        ar.append(value)          

In [ ]:
# 处理一系列图像数据，并将处理后的图像数据存储在 images 列表中，根据图像文件名中是否包含 "topimg" 来选择不同的处理方式
images = []
for i in ar:
    im=[]
    if "topimg" in i:
        im.append(proc(os.path.join('politifact_fake_top_img/',i)))
    else:
        im.append(proc(os.path.join('fake_img/',i)))
    images.append(im)

In [4]:
import keras
import tensorflow as tf
from keras import layers,models
import keras
import os
import gc

#installing vgg19
vgg16_model = keras.applications.vgg16.VGG16(weights='imagenet')
vgg16_model.summary() # 调用 vgg16_model.summary() 来查看模型的架构和参数数量
# type(vgg19_model) #not Sequential model 


from keras.models import Sequential
model = Sequential()
for layer in vgg16_model.layers[:-2]:
    model.add(layer)
model.summary() 
# from keras import optimizers
# adam = optimizers.Adam(lr = 1e-4)
# model.compile(adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
# last_layer = tf.keras.models.Model(inputs=vgg16_model.inputs, outputs=vgg16_model.layers[-2].output)


# em 字典包含了处理的图像文件名及其对应的预测结果或特征向量，而 os_error 列表包含了处理图像时出现异常的文件名。
os_error =[]
em ={}
t=0
ar = os.listdir('train')
n=500
final = [ar[i * n:(i + 1) * n] for i in range((len(ar) + n - 1) // n )]
for j in final:
    
    for i in j:
        print(i)
        try: 
            em[i] = model.predict(proc(os.path.join('train/',i)))
    
        except: 
            os_error.append(i)
        
#     gc.collect()

    
    

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
bloc

T4mzsLC392HbU7i5CBGXHibCXLmnEkid.jpg
t4O6kMORLkROKC1R28kzMOQeZtQJtE64.jpg
T4rudYl82mVkJLcFg5ChfkmkH7JngRcR.jpg
T4temB9pd7PU5R7bFlv5fTRxxeT1tOJc.jpg
T4uvB1dTttbinH29OJXD6hNTLy2kp437.jpg
T4z19wxL8qpfGX8w4lJ2cNdFYgMLuKN5.jpg
T51kldM8QcegJ9UxRJ1BMskVlr1eMO3T.jpg
t52oZC0FC1dJKSPKB2dsiZg3Mr0avsnO.jpg
t54ksk3bG79oveijNAwMkn50xWoZa95u.jpg
T56GgyjUMmqTSOrnUEuiZ9gBlYTX7JPK.jpg
T5Dub4bfkVr3gN8IVDxxWElmudW9rSTb.jpg
t5edKMt3nW3Ox4LV5stmRkiqRbO4Nou9.jpg
T5kWmEEn6trpAGetRrW8OxzVVNsWu7Tj.jpg
t5l6l4YIfrD774RCIlGtEN4uPL8khbiE.jpg
T5NldaN67dFrfmwZJHEpWKYRZuBTNw9S.jpg
T5XrJqBFN5Yje61IiYvodGBkg9WbI8sg.jpg
T5Z6EWz1mBG8A9muNIQqDoYl0oXQoAuw.jpg
T62LlQTzPjpDDzyilMWfljlhnM09LLFm.jpg
t63xcZIr00XwGjHOprIjrOQTxz85TAUG.jpg
T68UUKfJWxVtBDXjgV8ZanoSd6mMj9Cu.jpg
cIIgpcudKGaXfNlOa2EHvta2RafrvSrZ.jpg
cILrtM4m5nFmIoKOaTchOT9rBHuTSI0b.jpg
CIMHFQ4AlWLxdgvTygh4zxVE010cb5b6.jpg
CIMpU9WCngSKmXBw6VYtDTd37ipZJGOu.jpg
cINmaOrPsEX2moTiZuX042gcklmomxpX.jpg
cinvPGqgDUEGojDvZ1305Sv0KCMH7MoQ.jpg
ciujZ0faViufY5xwyT1hP3C5fank4mGU.jpg
c

KEddQODhb07VuBOG1BYTgVDijWllcJyP.jpg
KEdGlfQjBhOulIwTrF8Ylfo6d0Z0XLe4.jpg
kEDvkieOUDIPMGDvZIgki6Bev7kKG7IM.jpg
KeeP8zx5LxtRPh36bkIAG72ph5yZQHKt.jpg
KEmVR8omfYDkQxPDht6qDubQ8T4TQIlo.jpg
KeNVSlXPkOMw6rS4R4FnsIQpU9hG2qUA.jpg
KeoLsyL6AYQDIQHdwJNZvKXSJeEVJ9Ec.jpg
Keq9zJUrtInzAe2V9dzCREBmzc5MPdsy.jpg
kevgxVIwQLhRovortV254OJowSfiZfuv.jpg
KEXKw7HTt89qUcMLRdzy42lfTksFDC1f.jpg
keySo5rPuvBNfaqXfsQRf7l04WPSHea9.jpg
kEzgPkCj0O7fb27bkJPQslLtaIn9vyBt.jpg
4KxKWbiHdfbpRdFpBdD6Kj3cjhhb0yha.jpg
4L457m0kVAxsQjwv4AmVib3zvgtmSyaY.jpg
4L5A3SgCosEzVBSvsXhzY4gsE4XiyhcA.jpg
4LBpltjJwtMFRPaFzmyel4WQih8DtbWy.jpg
4LRRo8OW8nYuysU0J1rvLn7d7zmAjKNE.jpg
4M8XoVTzyia7Qtt6zjHh1HKg7MTebLRo.jpg
4MD45tYIUv8musj1JeILpxlmYapvj1fN.jpg
4mjx0SkWTlV6MVJ0t1m3VRCtEPe94sFn.jpg
4MODtvlPR6WPzbMsWgQ0R6coa8faPqZi.jpg
4MORW4Y30bJkCx0GCbmPf8VxIRvTZmiu.jpg
4mPNe4JYu1MMB7skrT2a4lEx3AXTsWgd.jpg
4mquwBwg6MwYO9DXKYsCGx1MrLYLegyz.jpg
4MSEfu8iq4aEuBdo45V4Scf2gnBddpAc.jpg
4muaHBEpDomEuc353G1ROLrfA5RbkmaI.jpg
4MzARYn3iitvmgsUwNBk8xZ7XrCFR5MK.jpg
4

kimxTD6HMQ7sOt34i82FGNPA9QIC01nP.jpg
kJCNLDjQXZ2JxHNFrdfMJInEXpRqeVMc.jpg
kk7edFOtMDZNuyEk2LbBRFVcjcj5bMYT.jpg
KKsJZA4dxVKVUn8P5aiJRSQc3UtRuUZY.jpg
kLPDxaixU3TttvmVJA0W7M7Yph3jFlrZ.jpg
kNcEFThgRUxaqTDYdtKWHx0FCNYniBmn.jpg
KOrC5GdyVOdxa6fiSYuIo6TGtqp5Ca2N.jpg
Kq0pMKi8lUHPbxc0SXJlWnPsz8zNWGe6.jpg
krH8zrFZ3yFXtbmYz5AfRH3PHRc6vj8P.jpg
kSHVpYGqZ0HfRWtAZ593EdHBiQzRN0QX.jpg
KtsfGSQQB4Dchlvo2iccSPCHUGgLJy90.jpg
kv8L8MyS1zz0BHJV9tjDdibMTcnudXAh.jpg
kwqQSvOPZMi7P4hxFwPyCIBetCXXADoq.jpg
kXmdBD5MXBnzOQiEBzuf3KCcF4zztfbU.jpg
KyzdDtd0IBI5vmpPsOR2v4CPI7eUnynk.jpg
dIQLO3WHMGwEqrmPwAsdWBLEVQyV9NWI.jpg
dIrtOZOj65G0exoLthbEKhNOZTwzqs7o.jpg
dIx1S6AqnNBgMDXsBmHfTs1Xa2FiYydF.jpg
Dj2Cf4bsFaWFpllmQRzlMd1OeTLY0E0p.jpg
DJ6I8wS6AN7JMZ2sfWwOkmc8mJ0hIBsB.jpg
dJBNSPoFwfzmuhduK5adgq5Q0jNkdA92.jpg
djfz4Ki027Q6xDSi80icgcaYB0K0iYSe.jpg
dJkxrSQfigtzKLj4lR6zYZlkqElk0oG3.jpg
DJkZHFxmpk92Zg6SJz7R5q9KXVUv7tB0.jpg
DjLdVQgbRdxDCVjMI8aeYZhUiAUD86RM.jpg
Djn21G1wCMQzE6zj2aNmp1d04TRBHdFJ.jpg
DjnWE7ZXKcDMcu0OjaiH9YjEEoFBPBwh.jpg
D

i8mKibq4KtLAfOcMH3gd1qhjF7xHTZvP.jpg
i9dUhjJNGS8QvxLiFc538XhP2okdTNpT.jpg
I9oPvcJ9ReIDq7JH3S13jCYAeJ9etHbo.jpg
i9U2GnlMiEjo8Qeg2sJTWw7oLFChTTQd.jpg
SiUapCvh78cCivZR340tiOne0K1vKS97.jpg
SiUickR4wbc1DsJAI4igQMyhaD8atCup.jpg
Sixvsqmfjfusirx8j25kLGssXC3iRU9s.jpg
SiztXZMu8wnriZmSjFhEPepOhPOygrQI.jpg
Sj0py1h7fnb94JdUtQ9UbIhAq7nmKytm.jpg
sJ1ghJhNFsSLLeZ8MPMVspMFUxm6YZv1.jpg
SJ1WJAU9H3SVsd5ZoOdGCYrAM4Ziof1p.jpg
SJ7fgwSClNhcHq1cYK56NDJlgq9nVjpk.jpg
Sjb3BQr5HeYhkMdBOZcix6e3xAc6PzGk.jpg
SjBY2aQhByEGSbcdxYkIJe7q7nfP7rrf.jpg
sjCndGxDTGwua8cgZynfh1DZcFng7cGv.jpg
sJcUgb3PLD4jIRuQeuWGxu9HWgF93FkB.jpg
sJcZErApkc12iBdggjeoT6M2LZZaJ2eb.jpg
sjjDae1cqHH173Ia6gt6MxiDDJjfKCll.jpg
YItw54W6LpTzyh24K1AKd31sa7bTVRKh.jpg
YJ1W67g3fSIRCqPCwV1Ko4FYknHlNpgT.jpg
YJ9Pa2FFTc4r6p7xaL7xNRAoy8Dxw2xc.jpg
YJd9xVyzDPDxXZpHKMyVmnU77zNzmWjc.jpg
yJioGwJSt3X3uqGzUlwojk2SiJ9m51vg.jpg
YJld5ONyBM2SSWZTKriOfEtQ3tarnK6z.jpg
YjLYj8HWcmUE11rxYX4JWJZkZhvC63ZM.jpg
yJN7jHOxUWvulyYWAhBw4Lg6RA8YqOqe.jpg
YJnYNNVkxtgvqGeI5yFKZR7pFRWDgpxF.jpg
Y

...

uMNXNTFVCryaopygxeztimDsUnDQCkNB.jpg
uMTqbXOWEZrZMxYQO28A0dVHuiYRCUth.jpg
UmUkSEMWSLWkiP3enwqXv0mxIi60d1Vy.jpg
Umw8wPvWeo50waVVaj1OJTVg86dnallH.jpg
umxBEHp1HzvnIGdHkwrR6QMad1G2ogvX.jpg
UMXGdUk8BBoGtHkb2CG8pH2rXISlMueG.jpg
UAgvejQjnpWX98ojS9vX1P4XZ7UfOvgG.jpg
UBK2RIhxM7jWWeRqCzNCWyeeGWVuRi5J.jpg
UCbaMSun4nbyLL1wKuL12Gozr6hYNklx.jpg
udlsuXgEcCeS1EYuxMCtCR3Wp3uJmRH2.jpg
UfaIGBcI2ggxAZiraUXnJm3JfqgWyDZs.jpg
UFYE0hKeJxa9o4KVBzFNjZA9DwF8iQNp.jpg
uhfSw5hAOJEQ2dQ88w4tq1I3XThGvYId.jpg
uInbeD7Iv5B60lss4Z5bwgllNU01MhsQ.jpg
uJjOr4QRaBvAOEPK7bSCLWzQ5aQckeYo.jpg
ukaOCUIILHCRGsNyDF8IM5Q7UOcVhiiO.jpg
ULseNnVL6npDwUdAUeiFKjkDhmrhYMj9.jpg
umeKueWs2otJ5IXBZQutkGR3r7bW3RUN.jpg
KaxeZDrUGQN5Tv4P2Yv8U5yVMdmMlTON.jpg
kAXOvCVg10PFOIqB3yNjzEobBn7GXWyL.jpg
KaxSqGP6JHn7zBW6AohOuzp9PFFB309n.jpg
KaxtW8K3m3R2bjmKiLjXoHyFQAvyvvyv.jpg
kaYrB01nWxoZyLeZhv8oW5XxecAGJwTF.jpg
KAZfShiKPe61CsOgVcRnIlQlJjKgmN2H.jpg
kb4ShBkz8jd9bjJpX4NhUfyJNHESCobj.jpg
kb5pALNONy32OuqU95rq2Yx3URx0aLYL.jpg
KbBKq5LaXn4rbOmcYYE6EbV9VXGwZJrw.jpg
k

PnVXRptWDbflTwqLHl4ySmGftE6MReaJ.jpg
pNZk38hOxWEEPS3Bo9CBTtoCeTjZwcky.jpg
pO0A9pDS957tVmD9aL3OmYCrctlm1fX8.jpg
po651sasTg0wMqzQYb94BXt4KetGsoTg.jpg
Po6Fm003C3Ci7U7OqDQa0xUwywvN5mf7.jpg
POAIEMOknPrgXQPB73u2WrwG5UgTJJCy.jpg
pOB2eH8XbF90EG95KxqhDvuZvA0QKYmu.jpg
pOC50DrWzQrIEuCQ87949qoxVqpLaLKY.jpg
poci0QGIrPZRm8EZOkwuFaacTAR3m1um.jpg
q1EIWiJTBU4bkw7baUi6og9RT9AaDLhO.jpg
q3Bv8L2NpzjHWuQlc7CiAv7qKw44IOMy.jpg
Q5hglGYvSSItawQ2NZvtAPal8f2thdkC.jpg
q8buxt0RE0wQeV5cBnm6OJCJ3p3C0Xws.jpg
QatEIKmUhPByGi1CPQgkR0NRZNbozq0p.jpg
qcLN4vUq6HmOOPJseIJ09QzKZjca8vdw.jpg
QdZE5ERDQHYgjLp6D3N1iSIgDsp4bfuR.jpg
qGNpYfWxC64xmTTye8XCpZqya2li3tR7.jpg
qIAvfEeFSQfVAcp9athYzQI3l528EpHf.jpg
QJrAaL5cL5jVKtccZftcEFaXTWnjZaw7.jpg
qKqOL8HbLBcQbBFPBljSeBQCEx33IqfB.jpg
H8xcKKig8hDmSxLpayPf6fOPzC837chu.jpg
h8Z0vdnQ2LGzGgvv957B9wRmJkIagClz.jpg
H8zWidaaUUzmAzNAMAPbCGbNJXA7Lwzr.jpg
H90kNdlG8r4mnnE65I1S43JanH1FPJj2.jpg
h9ds8KMuUBASsXV8B4d4OMh0kZfMEJYL.jpg
h9m4CvcEiORdGVlFYNQz2XT8SKaSB1LN.jpg
H9MAuwEHpjf9HKWtwfNGGcppSxWRNUwJ.jpg
h

oRTxgPd4nKTFIMxBF8ixuyW5g9f7j8jE.jpg
orv5gKfeGawrN8atyIOCqO3nNp4QdrL1.jpg
N7I2wNXzaF8zXGbVf5U2LuyCQzKbJnNR.jpg
N7jV4c3VtErh68PgpjCwJ0MTGcURjkNm.jpg
N7K0rXSEkzh5lOF9FbMbCBRTr7QD56kV.jpg
n7pjEWeKnB9FDaYFZFzO8FiwhSdNZfpq.jpg
n7stH5guvBLsdvw7VZkIdboKfLC7dIdo.jpg
n7uBng4onvL39eC9Hzi64GQuClzsJ1ES.jpg
N7XUPyA0CBrxghg9tOLY8UEnXzOJFjy1.jpg
N84mUe2z02U9J6yTLdXj6qNCM2KWAjXI.jpg
N888xMZd3j0k566kiOzNLPb7zvowCbq7.jpg
N889GJNIgNFJvbiU4Nn9eKNZef93eY8R.jpg
N8frELwI2R8pH8xhEoP7Q11bm7VynphA.jpg
n8os6Aa3g9dGyQ6Yln2E4rfDFyyYCoIM.jpg
n8vqqRmX3I2jzwAYbfMxcG5x6vxVuRPc.jpg
n8ziC63rXq68MEIN527vd0KYxv1KAo67.jpg
GimuDZ8zCthZY0foy2G3gPu96omdXzJg.jpg
gInxhHohBHYcuXSpTew6s4J0PY8PokdW.jpg
GioTgh2G7jTrLtqo8qljXX9tLieimE3g.jpg
GIOXMQ4qcHrh8teCxbeQa1eQqKWj9zYj.jpg
GIpIAPM9e4ZQ4uIQedqC2pzE92thG9qm.jpg
Gir4rJHisqrPH0lvdNwQ46QwMHsZDYr3.jpg
girJifHoimNQJox69tcJYQrp5vPSXfhb.jpg
git3494O6M444vYXMER5q33cm67PS5eJ.jpg
gIzaN0f4cfS355YehmbKoOp0XFwJYP7w.jpg
GizMiI3FaJ31TZyruVdlmC8hCrr2n1MO.jpg
gizuYbwSvEeUWLqyX9RKfzFQ9b4cigRY.jpg
G

jgHwON0DxDyoriV6YUK15BJl2WatvsMu.jpg
JGj84ioNLM0n5gu2fjGw3olwNUkm70zn.jpg
jgjxn3ya0WOjQXyT2A8ottT82Qy0WLKg.jpg
jgkfSj2exb8RS3UfQj5Aq5OzwQH0f43I.jpg
jGm4r8QDuR5NMq2Qz4JLnYDCVwf7QT6z.jpg
JgMzlfWlKBKH4UU3sv8auTpzzUKtYODA.jpg
jGoUEx5QM4jEWmr4glw8jk4SyZnryGWj.jpg
JgQFVcvGH5yZUh1wE33psn6iN1n4rCvg.jpg
jgR3h6UvRi2K907n8qG0B01RvmzHrXoF.jpg
JGr3qDizqp8LBA0Bt1fMjoLMFWkhpj2G.jpg
JgtvRCwtDJ7GDUvXkuNhXmjk12AdH7uG.jpg
jGtWj4Ei4ENLBKjV8ik029W7qNLcpKU4.jpg
Jh1bSenCK56vVUqObaCWKcZzFlElrwsW.jpg
Jh2uGMSSCRSbd8YMBOkuHXmCxeccxKlZ.jpg
jtOssS2vg9pjsPX30kgG7z4qmALkUEIF.jpg
JUT9oqwD5SXo7hChoHPANZO5EQeMCrHm.jpg
JvJXQrz6yK8t49yGaX1u1HZaRVl6ajU7.jpg
Jx0KogYgijmoYgAUmpZM1ZmX7XklgCXv.jpg
jYC1fWExxd74aeMAvjD29Ym0VZDT1swW.jpg
jznr9WcFki8GWz7MWrs7QeV6i5IYOfxU.jpg
K1VzDeIDZWqztgT6EEYMpyw1NdQMUoZz.jpg
K5UiMwkZKgdkKxv34KeK67zuGVpkjRGO.jpg
k97PdARZHhTIES3XQKHDLWTf2xe5KTM2.jpg
KaW1Z5RQqCaK9iE27fM55fdBgW7mvXR8.jpg
kbrYaD8OlyMb0UfdIjH5DpPfJMws4Wzl.jpg
kCU8XrscdUKAPYW43unDjZz40mBNuka2.jpg
2TDOviEHKhnXZ3uPQOKxlR9zwDTcQEd0.jpg
2

hgbYHoK0C1uzF4Jk4VWwFlTgOZKOgsZL.jpg
hGd0RbyzDRXbs3J6UKLyXxeUJZNSwVQa.jpg
Eb9LCYy0amN0BAlumfDQtDDIiIAMshvm.jpg
EBBJDHyiIS4vaw28wGVslxScLNo0GFKS.jpg
eBFIGY0k4iLjA8qdXAMW57gpXScQ6ai9.jpg
eBFnuljngj1RzQPpYw2cGC3NQnvvYRbG.jpg
EBjGpKGbcuyiBHRSPaDkKFGtfuT7GZSZ.jpg
EBk1AijWjcI9UQdkKvLOSkVP2dW6WDoM.jpg
EBlV5qKwA6aK0B8ZMppHLurdzbYl9TfX.jpg
ebSijpcSEzfGwusx2zilRAXeCblwBC0s.jpg
eBV5xXYmupxEec9mDcb2o4eWrxu71YkO.jpg
eBYCLqpXTcvcl8E4tepKQIQpzzjJOJh3.jpg
eC2fyUM2KHazzO34PnrZuL1PIxwAqtD3.jpg
EC3qXYEe3KdlPXoY1MGwPb56xd1VQtCF.jpg
echc8ti0IxgMS5GehaQDElKtRlokI9pp.jpg
fsSoUXXg0XY1aeSH2dYa8i9GUZzzXA43.jpg
FStgUMrBQpcNaRsBtnEoKDK5ymy6Le7n.jpg
FT0xI0G9UQzdvGuPTyVAMGy4v8NwD0YS.jpg
ft36cSaUZIuHDUdp1FjKvn8JNK27DgSp.jpg
fT5NaDUGwSVIUV08gktJWzSjpFt5qKXY.jpg
FT7I1cKONSSd9n76ZZtu0FmcYp9oVINP.jpg
Ft7MfE6JOlMZlZ0VcMbTZQonGG7AQumx.jpg
Ft8rbNT7w3b3IGM1zUpGKIxcn7LfGW3l.jpg
ft9JPBGcV4Uf20NjlKWt61ZeHfyyTRRd.jpg
FtaOlr7PulLr0Zxl7LWbvhRd4iersiv7.jpg
FTaQ9HCEJVEcbELx4Or6mB3IjpnvR5q0.jpg
FtD8cJH6TQGMLcrJ8dCqwUNkzeNqIGsx.jpg
J

In [5]:
with open('train_gossip.pickle', 'wb') as handle:
    pickle.dump(em, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
with open('train_gossip.pickle', 'rb') as handle:
    b = pickle.load(handle)
